## Variational Autoencoder (VAE)

The Variational Autoencoder (VAE) is a generative deep learning model proposed by Kingma and Welling in 2013. It is a variant of the Autoencoder but incorporates the concept of Variational Inference, enabling it to generate new data rather than merely compressing and reconstructing inputs. The primary goal of VAE is to learn a latent representation of data and generate samples similar to the training data by sampling from the latent space.

### Core Components of VAE Include:
- **Encoder**: Maps input data x to the distribution parameters of the latent space (typically the mean μ and variance σ² of a Gaussian distribution).
- **Sampling**: Samples latent variables z from the latent distribution using the reparameterization trick to make the sampling process differentiable.
- **Decoder**: Reconstructs output data x' from the latent variable z, aiming to make x' as close as possible to x.
- **Loss Function**: Combines reconstruction loss (e.g., MSE) and KL divergence (Kullback-Leibler divergence) to regularize the latent distribution, making it close to a prior distribution (typically a standard normal distribution).

The advantage of VAE lies in its ability to create a continuous latent space, supporting interpolation and generating new samples. It is commonly used in image generation, data augmentation, and other fields. Compared to GANs (Generative Adversarial Networks), VAE training is more stable, but the generated samples may be blurrier.

![Figure](https://github.com/user-attachments/assets/d8b5e82e-5b83-41d9-8b3c-521a3aeeb38e)
### Mathematical Description
The goal of VAE is to maximize the marginal likelihood $p(x)$, which is typically intractable to compute directly. Therefore, the Evidence Lower Bound (ELBO) is used as a proxy optimization objective. Assumptions:
- Prior distribution: $p(z) = N(0, I)$ (standard normal distribution).
- Approximate posterior: $q(z|x) = N(\mu, \sigma^2I)$, parameterized by the encoder, where $\mu$ and $\sigma$ are computed from x by a neural network.
- Generative model: $p(x|z)$, parameterized by the decoder, typically assumed as $p(x|z) = N(\text{decoder output}, I)$ or a Bernoulli distribution (for binary data).

The ELBO is mathematically expressed as:

$$
\mathcal{L}(\theta, \phi; x) = \mathbb{E}_{q_\phi(z|x)} [\log p_\theta(x|z)] - D_{KL}(q_\phi(z|x) \| p(z))
$$

Where:
- $\theta$ represents the decoder parameters, and $\phi$ represents the encoder parameters.
- The first term is the reconstruction loss: measures the accuracy of reconstructing x from z, typically implemented as negative log-likelihood (e.g., MSE for continuous data: $\|x - \hat{x}\|^2 / 2$).
- The second term is the KL divergence: regularizes $q(z|x)$ to be close to $p(z)$, with the formula (assuming Gaussian distribution):

$$
D_{KL}(q(z|x) \| p(z)) = -\frac{1}{2} \sum_{j=1}^J (1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2)
$$

Where $J$ is the dimension of the latent space.

To enable gradient propagation, the reparameterization trick is used: $z = \mu + \sigma \odot \epsilon$, where $\epsilon \sim N(0, I)$.

Optimization process: Maximize the ELBO (equivalent to minimizing the negative ELBO) using stochastic gradient descent.

## Implementation Notes
The following is a minimal VAE implementation using PyTorch for the MNIST dataset (28x28 grayscale images). It uses a simple multilayer perceptron (MLP) as the encoder and decoder, with a latent dimension of 2 (for visualization purposes). The code is consolidated into a single module, including model definition, loss function, training loop, and sample generation. Running it requires PyTorch and torchvision (`pip install torch torchvision`).

- **Runtime Environment**: Ensure GPU support for faster training (the code automatically detects the device).
- **Extensions**: This is a simplified version for understanding VAE principles. In practice, convolutional neural networks (CNNs) can replace MLPs, the latent dimension can be increased, or hyperparameters can be tuned for better performance.
- **Sample Generation**: After training, uncomment the `save_image` section to save generated MNIST image samples.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set hyperparameters
input_dim = 28 * 28  # MNIST image size
hidden_dim = 400
latent_dim = 2  # Latent space dimension
batch_size = 128
epochs = 10
lr = 1e-3

# Data loading
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))  # Output in [0,1], suitable for MNIST

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')  # Reconstruction loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  # KL divergence
    return BCE + KLD

# Main function: Training and generation
def main():
    # Initialize model and optimizer
    model = VAE()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {train_loss / len(train_loader.dataset):.4f}')

    # Generate samples
    with torch.no_grad():
        z = torch.randn(64, latent_dim)  # Random sampling
        samples = model.decode(z).view(64, 1, 28, 28)
        # Uncomment the following to save generated images
        # from torchvision.utils import save_image
        # save_image(samples, 'samples.png')

if __name__ == "__main__":
    main()